# US Bike Share

## Table of Contents
- [Introduction](#intro)
- [Posing Questions](#pose_questions)
- [Data Collection and Wrangling](#wrangling)
  - [Condensing the Trip Data](#condensing)
- [Exploratory Data Analysis](#eda)
  - [Statistics](#statistics)
  - [Visualizations](#visualizations)
- [Performing Your Own Analysis](#eda_continued)
- [Conclusions](#conclusions)

<a id='intro'></a>
## Introduction

Over the past decade, bicycle-sharing systems have been growing in number and popularity in cities across the world. Bicycle-sharing systems allow users to rent bicycles for short trips, typically 30 minutes or less. Thanks to the rise in information technologies, it is easy for a user of the system to access a dock within the system to unlock or return bicycles. These technologies also provide a wealth of data that can be used to explore how these bike-sharing systems are used.

In this project, you will perform an exploratory analysis on data provided by [Motivate](https://www.motivateco.com/), a bike-share system provider for many major cities in the United States. You will compare the system usage between three large cities: New York City, Chicago, and Washington, DC. You will also see if there are any differences within each system for those users that are registered, regular users and those users that are short-term, casual users.

<a id='pose_questions'></a>
## Posing Questions

Before looking at the bike sharing data, you should start by asking questions you might want to understand about the bike share data. Consider, for example, if you were working for Motivate. What kinds of information would you want to know about in order to make smarter business decisions? If you were a user of the bike-share service, what factors might influence how you would want to use the service?

**Question 1**: Write at least two questions related to bike sharing that you think could be answered by data.

**Answer**: 
- First question is that in which month the users uses the bike most?. We can give discount to the user according to this.
- what is the most popular hour / weekday for three city ?


<a id='wrangling'></a>
## Data Collection and Wrangling

Now it's time to collect and explore our data. In this project, we will focus on the record of individual trips taken in 2016 from our selected cities: New York City, Chicago, and Washington, DC. Each of these cities has a page where we can freely download the trip data.:

- New York City (Citi Bike): [Link](https://www.citibikenyc.com/system-data)
- Chicago (Divvy): [Link](https://www.divvybikes.com/system-data)
- Washington, DC (Capital Bikeshare): [Link](https://www.capitalbikeshare.com/system-data)

If you visit these pages, you will notice that each city has a different way of delivering its data. Chicago updates with new data twice a year, Washington DC is quarterly, and New York City is monthly. **However, you do not need to download the data yourself.** The data has already been collected for you in the `/data/` folder of the project files. While the original data for 2016 is spread among multiple files for each city, the files in the `/data/` folder collect all of the trip data for the year into one file per city. Some data wrangling of inconsistencies in timestamp format within each city has already been performed for you. In addition, a random 2% sample of the original data is taken to make the exploration more manageable. 

**Question 2**: However, there is still a lot of data for us to investigate, so it's a good idea to start off by looking at one entry from each of the cities we're going to analyze. Run the first code cell below to load some packages and functions that you'll be using in your analysis. Then, complete the second code cell to print out the first trip recorded from each of the cities (the second line of each data file).

In [ ]:
## import all necessary packages and functions.
import csv # read and write csv files
from datetime import datetime # operations to parse dates
from pprint import pprint # use to print data structures like dictionaries in
                          # a nicer way than the base print function.
import seaborn as sns

In [ ]:
def print_first_point(filename):
    """
    This function prints and returns the first data point (second row) from
    a csv file that includes a header row.
    """
    # print city name for reference
    city = filename.split('-')[0].split('/')[-1]
    print('\nCity: {}'.format(city))
    
    with open(filename, 'r') as f_in:
        ## TODO: Use the csv library to set up a DictReader object. ##
        ## see https://docs.python.org/3/library/csv.html           ##
        trip_reader =csv.DictReader(f_in)
        
        ## TODO: Use a function on the DictReader object to read the     ##
        ## first trip from the data file and store it in a variable.     ##
        ## see https://docs.python.org/3/library/csv.html#reader-objects ##
        first_trip = next(trip_reader)
        
        ## TODO: Use the pprint library to print the first trip. ##
        ## see https://docs.python.org/3/library/pprint.html     ##
        pprint(first_trip)
        
    # output city name and first trip for later testing
    return (city, first_trip)

# list of files for each city
data_files = ['../input/NYC-CitiBike-2016.csv',
              '../input/Chicago-Divvy-2016.csv',
              '../input/Washington-CapitalBikeshare-2016.csv',]

# print the first trip from each file, store in dictionary
example_trips = {}
for data_file in data_files:
    city, first_trip = print_first_point(data_file)
    example_trips[city] = first_trip

If everything has been filled out correctly, you should see below the printout of each city name (which has been parsed from the data file name) that the first trip has been parsed in the form of a dictionary. When you set up a `DictReader` object, the first row of the data file is normally interpreted as column names. Every other row in the data file will use those column names as keys, as a dictionary is generated for each row.

This will be useful since we can refer to quantities by an easily-understandable label instead of just a numeric index. For example, if we have a trip stored in the variable `row`, then we would rather get the trip duration from `row['duration']` instead of `row[0]`.

<a id='condensing'></a>
### Condensing the Trip Data

It should also be observable from the above printout that each city provides different information. Even where the information is the same, the column names and formats are sometimes different. To make things as simple as possible when we get to the actual exploration, we should trim and clean the data. Cleaning the data makes sure that the data formats across the cities are consistent, while trimming focuses only on the parts of the data we are most interested in to make the exploration easier to work with.

You will generate new data files with five values of interest for each trip: trip duration, starting month, starting hour, day of the week, and user type. Each of these may require additional wrangling depending on the city:

- **Duration**: This has been given to us in seconds (New York, Chicago) or milliseconds (Washington). A more natural unit of analysis will be if all the trip durations are given in terms of minutes.
- **Month**, **Hour**, **Day of Week**: Ridership volume is likely to change based on the season, time of day, and whether it is a weekday or weekend. Use the start time of the trip to obtain these values. The New York City data includes the seconds in their timestamps, while Washington and Chicago do not. The [`datetime`](https://docs.python.org/3/library/datetime.html) package will be very useful here to make the needed conversions.
- **User Type**: It is possible that users who are subscribed to a bike-share system will have different patterns of use compared to users who only have temporary passes. Washington divides its users into two types: 'Registered' for users with annual, monthly, and other longer-term subscriptions, and 'Casual', for users with 24-hour, 3-day, and other short-term passes. The New York and Chicago data uses 'Subscriber' and 'Customer' for these groups, respectively. For consistency, you will convert the Washington labels to match the other two.


**Question 3a**: Complete the helper functions in the code cells below to address each of the cleaning tasks described above.

In [ ]:
def duration_in_mins(datum, city):
    """
    Takes as input a dictionary containing info about a single trip (datum) and
    its origin city (city) and returns the trip duration in units of minutes.
    
    Remember that Washington is in terms of milliseconds while Chicago and NYC
    are in terms of seconds. 
    
    HINT: The csv module reads in all of the data as strings, including numeric
    values. You will need a function to convert the strings into an appropriate
    numeric type when making your transformations.
    see https://docs.python.org/3/library/functions.html
    """
    if city=="Washington":
        duration = (float(datum['Duration (ms)'])*(0.001))/60
    else:
        duration = (float(datum['tripduration']))/60
    # YOUR CODE HERE
    
    return duration


# Some tests to check that your code works. There should be no output if all of
# the assertions pass. The `example_trips` dictionary was obtained from when
# you printed the first trip from each of the original data files.
tests = {'NYC': 13.9833,
         'Chicago': 15.4333,
         'Washington': 7.1231}

for city in tests:
    assert abs(duration_in_mins(example_trips[city], city) - tests[city]) < .001

In [ ]:
def time_of_trip(datum, city):
    """
    Takes as input a dictionary containing info about a single trip (datum) and
    its origin city (city) and returns the month, hour, and day of the week in
    which the trip was made.
    
    Remember that NYC includes seconds, while Washington and Chicago do not.
    
    HINT: You should use the datetime module to parse the original date
    strings into a format that is useful for extracting the desired information.
    see https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    """
    
    # YOUR CODE HERE
    if city=="NYC":
        starttime = datum['starttime']
        s1 = datetime.strptime(starttime, '%m/%d/%Y %H:%M:%S')
        return(s1.month,s1.hour,s1.strftime('%A'))
    
    elif city=="Washington":
        starttime = datum['Start date']
        s1 = datetime.strptime(starttime, '%m/%d/%Y %H:%M')
        return(s1.month,s1.hour,s1.strftime('%A'))
    
    elif city=="Chicago":
        starttime = datum['starttime']
        s1 = datetime.strptime(starttime, '%m/%d/%Y %H:%M')
        return(s1.month,s1.hour,s1.strftime('%A'))
    


# Some tests to check that your code works. There should be no output if all of
# the assertions pass. The `example_trips` dictionary was obtained from when
# you printed the first trip from each of the original data files.
tests = {'NYC': (1, 0, 'Friday'),
         'Chicago': (3, 23, 'Thursday'),
         'Washington': (3, 22, 'Thursday')}

for city in tests:
    assert time_of_trip(example_trips[city], city) == tests[city]

In [ ]:
def type_of_user(datum, city):
    """
    Takes as input a dictionary containing info about a single trip (datum) and
    its origin city (city) and returns the type of system user that made the
    trip.
    
    Remember that Washington has different category names compared to Chicago
    and NYC. 
    """
    
    # YOUR CODE HERE
    if city=="NYC" or city=="Chicago":
        user_type = datum['usertype']
    elif city=="Washington":
        if datum['Member Type']=="Registered":
            user_type = "Subscriber"
        else:
            user_type = "Customer"
    
    return user_type


# Some tests to check that your code works. There should be no output if all of
# the assertions pass. The `example_trips` dictionary was obtained from when
# you printed the first trip from each of the original data files.
tests = {'NYC': 'Customer',
         'Chicago': 'Subscriber',
         'Washington': 'Subscriber'}

for city in tests:
    assert type_of_user(example_trips[city], city) == tests[city]

**Question 3b**: Now, use the helper functions you wrote above to create a condensed data file for each city consisting only of the data fields indicated above. In the `/examples/` folder, you will see an example datafile from the [Bay Area Bike Share](http://www.bayareabikeshare.com/open-data) before and after conversion. Make sure that your output is formatted to be consistent with the example file.

In [ ]:
def condense_data(in_file, out_file, city):
    """
    This function takes full data from the specified input file
    and writes the condensed data to a specified output file. The city
    argument determines how the input file will be parsed.
    
    HINT: See the cell below to see how the arguments are structured!
    """
    
    with open(out_file, 'w') as f_out, open(in_file, 'r') as f_in:
        # set up csv DictWriter object - writer requires column names for the
        # first row as the "fieldnames" argument
        out_colnames = ['duration', 'month', 'hour', 'day_of_week', 'user_type']        
        trip_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
        trip_writer.writeheader()
        
        ## TODO: set up csv DictReader object ##
        trip_reader = csv.DictReader(f_in)

        # collect data from and process each row
        for row in trip_reader:
            # set up a dictionary to hold the values for the cleaned and trimmed
            # data point
            new_point = {}

            ## TODO: use the helper functions to get the cleaned data from  ##
            ## the original data dictionaries.                              ##
            ## Note that the keys for the new_point dictionary should match ##
            ## the column names set in the DictWriter object above.         ##
            new_point['duration'] = duration_in_mins(row,city)
            new_point['month'],new_point['hour'],new_point['day_of_week'] = time_of_trip(row,city)
            new_point['user_type'] = type_of_user(row,city)

            ## TODO: write the processed information to the output file.     ##
            ## see https://docs.python.org/3/library/csv.html#writer-objects ##
            f_out.write(str(new_point['duration']))
            f_out.write(",")
            f_out.write(str(new_point['month']))
            f_out.write(",")
            f_out.write(str(new_point['hour']))
            f_out.write(",")
            f_out.write(str(new_point['day_of_week']))
            f_out.write(",")
            f_out.write(str(new_point['user_type']))
            f_out.write("\n")      

In [ ]:
# Generate summary files for each of the cities
city_info = {'Washington': {'in_file': '../input/Washington-CapitalBikeshare-2016.csv',
                            'out_file': '../working/Washington-2016-Summary.csv'},
             'Chicago': {'in_file': '../input/Chicago-Divvy-2016.csv',
                         'out_file': '../working/Chicago-2016-Summary.csv'},
             'NYC': {'in_file': '../input/NYC-CitiBike-2016.csv',
                     'out_file': '../working/NYC-2016-Summary.csv'}}

for city, filenames in city_info.items():
    condense_data(filenames['in_file'], filenames['out_file'], city)
    print_first_point(filenames['out_file'])

<a id='eda'></a>
## Exploratory Data Analysis

Now that you have the data collected and wrangled, you're ready to start exploring the data. In this section you will write some code to compute descriptive statistics from the data. You will also be introduced to the `matplotlib` library to create some basic histograms of the data.

<a id='statistics'></a>
### Statistics

First, let's compute some basic counts. The first cell below contains a function that uses the csv module to iterate through a provided data file, returning the number of trips made by subscribers and customers. The second cell runs this function on the example Bay Area data in the `/examples/` folder. Modify the cells to answer the question below.

**Question 4a**: Which city has the highest number of trips? Which city has the highest proportion of trips made by subscribers? Which city has the highest proportion of trips made by short-term customers?

**Answer**: 
- **NYC** city has the highest number of trips **(276798)**. 
- **NYC** city has the highest proportion of trips made by subscribers**(0.88)** 
- **Chicago** city has the highest proportion of trips made by short-term customers **(0.23)**.

In [ ]:
def number_of_trips(filename):
    """
    This function reads in a file with trip data and reports the number of
    trips made by subscribers, customers, and total overall.
    """
    with open(filename, 'r') as f_in:
        # set up csv reader object
        reader = csv.DictReader(f_in)
        
        # initialize count variables
        n_subscribers = 0
        n_customers = 0
        
        # tally up ride types
        for row in reader:
            if row['user_type'] == 'Subscriber':
                n_subscribers += 1
            else:
                n_customers += 1
        
        # compute total number of rides
        n_total = n_subscribers + n_customers
        
        # return tallies as a tuple
        return(n_subscribers, n_customers, n_total)

In [ ]:
## Modify this and the previous cell to answer Question 4a. Remember to run ##
## the function on the cleaned data files you created from Question 3.      ##

data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    n_subscribers,n_customers,n_total = number_of_trips(datafile)
    sub_p = (n_subscribers / n_total)*100
    cus_p = (n_customers / n_total)*100
    # print city name for reference
    city = datafile.split('-')[0].split('/')[-1]
    print('In ' + city +' city, the total no. of Subscribers are ' + str(n_subscribers) +' ('+ str(abs(sub_p)) +'%)' + ', Customers are ' + str(n_customers)+' ('+ str(abs(cus_p)) +'%)' + ' and total no. of rides is ' + str(n_total) + '\n')

#data_file = './examples/BayArea-Y3-Summary.csv'
#print(number_of_trips(data_file))

Now, you will write your own code to continue investigating properties of the data.

**Question 4b**: Bike-share systems are designed for riders to take short trips. Most of the time, users are allowed to take trips of 30 minutes or less with no additional charges, with overage charges made for trips of longer than that duration. What is the average trip length for each city? What proportion of rides made in each city are longer than 30 minutes?

**Answer**: 
- The Average Trip Length For Each City:<br>
  NYC &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;: 15.81 Minutes <br>
  Washington&nbsp;: 18.93 Minutes <br>
  Chicago &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; : 16.56 Minutes <br>

- Proportion Of rides made in each city longer than 30 minutes: <br>
  NYC &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;: 0.073 <br>
  Washington&nbsp;: 0.108 <br>
  Chicago &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; : 0.083 <br>


In [ ]:
## Use this and additional cells to answer Question 4b.                 ##
##                                                                      ##
## HINT: The csv module reads in all of the data as strings, including  ##
## numeric values. You will need a function to convert the strings      ##
## into an appropriate numeric type before you aggregate data.          ##
## TIP: For the Bay Area example, the average trip length is 14 minutes ##
## and 3.5% of trips are longer than 30 minutes.                        ##

def trip_length(filename):
    total_ride = 0
    s = 0
    t_time = 0
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        for row in reader:
            total_ride = total_ride + 1
            duration = float(row['duration'])
            t_time = t_time + duration
            if duration>30:
                s = s + 1
            
        avg = (t_time/total_ride)
        p = (s/total_ride)
        
        return (t_time,total_ride,avg,p)

In [ ]:
data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    t_time,total,avg,p  = trip_length(datafile)
    # print city name for reference
    city = datafile.split('-')[0].split('/')[-1]
    print("In "+city+" City: ")
    print("The Average Trip Length Is: "+ str(avg))
    print("Proportion of rides longer than 30 min: "+ str(p)+"\n")
    

**Question 4c**: Dig deeper into the question of trip duration based on ridership. Choose one city. Within that city, which type of user takes longer rides on average: Subscribers or Customers?

**Answer**: In **Newyork** city, Customer takes longer ride on average **32.77 Minutes** and Subscribers takes longer rides on average **13.68 Minutes** in the same city. 

In [ ]:
## Use this and additional cells to answer Question 4c. If you have    ##
## not done so yet, consider revising some of your previous code to    ##
## make use of functions for reusability.                              ##
##                                                                     ##
## TIP: For the Bay Area example data, you should find the average     ##
## Subscriber trip duration to be 9.5 minutes and the average Customer ##
## trip duration to be 54.6 minutes. Do the other cities have this     ##
## level of difference?                                                ##
def ridership(filename):
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        s = 0
        c = 0
        s_duration = 0
        c_duration = 0
        for row in reader:
            user_type = row['user_type']
            duration = float(row['duration'])
            if user_type == 'Subscriber':
                s = s + 1
                s_duration = s_duration + duration
            else:
                c = c + 1
                c_duration = c_duration + duration
            
        s_avg = s_duration/s
        c_avg = c_duration/c
        return (s,c,s_avg,c_avg)


In [ ]:
data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    s,c,s_avg,c_avg  = ridership(datafile)
    # print city name for reference
    city = datafile.split('-')[0].split('/')[-1]
    print("In "+city+" City: ")
    print("Average Ride Taken By Subscribers: "+ str(s_avg))
    print("Average Ride Taken By Customers  : "+ str(c_avg)+"\n")
    

<a id='visualizations'></a>
### Visualizations

The last set of values that you computed should have pulled up an interesting result. While the mean trip time for Subscribers is well under 30 minutes, the mean trip time for Customers is actually _above_ 30 minutes! It will be interesting for us to look at how the trip times are distributed. In order to do this, a new library will be introduced here, `matplotlib`. Run the cell below to load the library and to generate an example plot.

In [ ]:
# load library
import matplotlib.pyplot as plt

# this is a 'magic word' that allows for plots to be displayed
# inline with the notebook. If you want to know more, see:
# http://ipython.readthedocs.io/en/stable/interactive/magics.html
%matplotlib inline 

# example histogram, data taken from bay area sample
data = [ 7.65,  8.92,  7.42,  5.50, 16.17,  4.20,  8.98,  9.62, 11.48, 14.33,
        19.02, 21.53,  3.90,  7.97,  2.62,  2.67,  3.08, 14.40, 12.90,  7.83,
        25.12,  8.30,  4.93, 12.43, 10.60,  6.17, 10.88,  4.78, 15.15,  3.53,
         9.43, 13.32, 11.72,  9.85,  5.22, 15.10,  3.95,  3.17,  8.78,  1.88,
         4.55, 12.68, 12.38,  9.78,  7.63,  6.45, 17.38, 11.90, 11.52,  8.63,]
plt.hist(data)
plt.title('Distribution of Trip Durations')
plt.xlabel('Duration (m)')
plt.show()
sns.set_style('whitegrid')

In the above cell, we collected fifty trip times in a list, and passed this list as the first argument to the `.hist()` function. This function performs the computations and creates plotting objects for generating a histogram, but the plot is actually not rendered until the `.show()` function is executed. The `.title()` and `.xlabel()` functions provide some labeling for plot context.

You will now use these functions to create a histogram of the trip times for the city you selected in question 4c. Don't separate the Subscribers and Customers for now: just collect all of the trip times and plot them.

In [ ]:
## Use this and additional cells to collect all of the trip times as a list ##
## and then use pyplot functions to generate a histogram of trip times.     ##
def plot_trip(filename):
    #totaltime,total_trip,avg,p = trip_length(filename)
    data = []
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        for row in reader:
            data.append(float(row['duration']))
            #plt.hist(reader)
        plt.hist(data)
        city = datafile.split('-')[0].split('/')[-1]
        plt.title(city)
        plt.xlabel("Trip Duration")
        plt.ylabel("No of Users")
        plt.show()

In [ ]:
data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    plot_trip(datafile)

In [ ]:
#Again plot the graph for different cities
#change the bin size
import numpy as py
def plot_trip_again(filename):
    #totaltime,total_trip,avg,p = trip_length(filename)
    data = []
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        for row in reader:
            data.append(float(row['duration']))
            #plt.hist(reader)
        bins = py.arange(0,100,5)
        plt.hist(data,bins)
        city = datafile.split('-')[0].split('/')[-1]
        plt.title(city)
        plt.xlabel("Trip Duration")
        plt.ylabel("No of Users")
        plt.show()

In [ ]:
data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    plot_trip_again(datafile)

If you followed the use of the `.hist()` and `.show()` functions exactly like in the example, you're probably looking at a plot that's completely unexpected. The plot consists of one extremely tall bar on the left, maybe a very short second bar, and a whole lot of empty space in the center and right. Take a look at the duration values on the x-axis. This suggests that there are some highly infrequent outliers in the data. Instead of reprocessing the data, you will use additional parameters with the `.hist()` function to limit the range of data that is plotted. Documentation for the function can be found [[here]](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.hist.html#matplotlib.pyplot.hist).

**Question 5**: Use the parameters of the `.hist()` function to plot the distribution of trip times for the Subscribers in your selected city. Do the same thing for only the Customers. Add limits to the plots so that only trips of duration less than 75 minutes are plotted. As a bonus, set the plots up so that bars are in five-minute wide intervals. For each group, where is the peak of each distribution? How would you describe the shape of each distribution?

**Answer**: Since you said to plot for one city, i choose to plot of **Chicago** city. For Subscriber group, the peak value of trip distribution is **5-10** Minutes and for Customer group, the peak of the trip distribution is **20-25** Minutes. As we can see that the distribution of trip duration in chicago city for customers is more likely to be **Normal Distribution(Bell-Shaped)**. And the trip duration for Subscribers is **positively skewed**, with more point on the right side of the graph.


In [ ]:
## Use this and additional cells to answer Question 5. ##
## Plot a graph 
def plot_cus(filename):
    data1 = []
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        for row in reader:
            if row['user_type']=='Customer':
                if float(row['duration'])<75:
                    data1.append(float(row['duration']))
        
        bins = py.arange(0,80,5)
        plt.hist(data1,bins,histtype='bar',rwidth=0.8)
        plt.xticks(bins)
        city = datafile.split('-')[0].split('/')[-1]
        plt.title("Distribution of Trip Durations for Customer in " + city)
        plt.xlabel("Trip Duration")
        plt.ylabel("No of Users")
        plt.show()

In [ ]:
data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    plot_cus(datafile)

<a id='eda_continued'></a>
## Performing Your Own Analysis

So far, you've performed an initial exploration into the data available. You have compared the relative volume of trips made between three U.S. cities and the ratio of trips made by Subscribers and Customers. For one of these cities, you have investigated differences between Subscribers and Customers in terms of how long a typical trip lasts. Now it is your turn to continue the exploration in a direction that you choose. Here are a few suggestions for questions to explore:

- How does ridership differ by month or season? Which month / season has the highest ridership? Does the ratio of Subscriber trips to Customer trips change depending on the month or season?
- Is the pattern of ridership different on the weekends versus weekdays? On what days are Subscribers most likely to use the system? What about Customers? Does the average duration of rides change depending on the day of the week?
- During what time of day is the system used the most? Is there a difference in usage patterns for Subscribers and Customers?

If any of the questions you posed in your answer to question 1 align with the bullet points above, this is a good opportunity to investigate one of them. As part of your investigation, you will need to create a visualization. If you want to create something other than a histogram, then you might want to consult the [Pyplot documentation](https://matplotlib.org/devdocs/api/pyplot_summary.html). In particular, if you are plotting values across a categorical variable (e.g. city, user type), a bar chart will be useful. The [documentation page for `.bar()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.bar.html#matplotlib.pyplot.bar) includes links at the bottom of the page with examples for you to build off of for your own use.

**Question 6**: Continue the investigation by exploring another question that could be answered by the data available. Document the question you want to explore below. Your investigation should involve at least two variables and should compare at least two groups. You should also use at least one visualization as part of your explorations.

**Question**: How does ridership differ by month? Which month has the highest ridership? Does the ratio of Subscriber trips to Customer trips change depending on the month?

**Answer**:  Since you said to investigate for one city, i choose the Washington City.
- In Washington City:<br>
  The firts bar chart shows the total number of riders in washington city in each month. According to this bar chart we can say   that highest number of users are in the month of june and july. <br>
  The second bar chart shows the total number of subscribers and customers in each month. According to this bar chart we see       more Subscribers than Customers in each of the month.<br>
  The third bar chart shows the ratio of Subscriber trip to Customer trip change depending on the month.According to this we can   say that the ratio between them is higher in the month of January, Febuary and December and in the month of May, June and July. 

In [ ]:
## Use this and additional cells to continue to explore the dataset. ##
## Once you have performed your exploration, document your findings  ##
## in the Markdown cell above.                                       ##
def monthly_analysis(filename):
    month_trip_sub={}
    month_trip_cus={}
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        data=[]
        data1=[]
        for i in range(1,13):
            month_trip_sub[str(i)]=0
            month_trip_cus[str(i)]=0 
        for row in reader:
            if row['user_type']=='Subscriber':
                month_trip_sub[row['month']]+=1
            else:
                month_trip_cus[row['month']]+=1
        data = month_trip_sub.values()
        data1 = month_trip_cus.values()
        bins = py.arange(1,13,1)
        plt.bar(bins+0,data,width=0.4,label ='Subscriber')
        plt.bar(bins+.40,data1,width=0.4,label ='Customer')
        city = datafile.split('-')[0].split('/')[-1]
        plt.title("Monthly Analysis Of City " + city + " (Subscribers/Customer)")
        plt.xticks(bins,['Jan','Feb','Mar','Apr','may','Jun','July','Aug','Sep','Oct','Nov','Dec'])
        plt.xlabel("Month")
        plt.ylabel("No of Users")
        plt.legend()
        plt.show()

In [ ]:
##Count Total Trips in Each Month and plot a bar graph
def total_trips_monthly(filename):
    total_trip_monthly = []
    total={}
    for i in range(1,13):
        total[str(i)]=0
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        for row in reader:
            total[row['month']]+=1
            #total_trip_monthly.append(int(row['month']))
        total_trip_monthly = total.values()
        bins = py.arange(1,13,1)
        #plt.hist(total_trip_monthly,bins,histtype='bar',rwidth=0.8)
        plt.bar(bins,total_trip_monthly,width=0.9)
        city = datafile.split('-')[0].split('/')[-1]
        plt.title("Monthly Analysis Of City " + city+ " (Total Users)")
        plt.xticks(bins,['Jan','Feb','Mar','Apr','may','Jun','July','Aug','Sep','Oct','Nov','Dec'])
        plt.xlabel("Month")
        plt.ylabel("No of Users")
        plt.show()
    

In [ ]:
##Calculate ratio of subscriber and customer in each month
def ratio(filename):
    month_trip_sub={}
    month_trip_cus={}
    with open(filename,'r') as fin:
        reader = csv.DictReader(fin)
        data=[]
        data1=[]
        data2=[]
        for i in range(1,13):
            month_trip_sub[str(i)]=0
            month_trip_cus[str(i)]=0 
        for row in reader:
            if row['user_type']=='Subscriber':
                month_trip_sub[row['month']]+=1
            else:
                month_trip_cus[row['month']]+=1
        for i in range(1,13):
            data.append(month_trip_sub[str(i)])
            data1.append(month_trip_cus[str(i)])
        for j in range(12):
            data2.append(data[j]/data1[j])
        bins = py.arange(1,13,1)
        #plt.hist(data2,bins,histtype='bar',rwidth=0.8)
        plt.bar(bins,data2,width=0.6)
        city = datafile.split('-')[0].split('/')[-1]
        plt.title("Ratio Of (Subscriber/Customer) in " + city)
        plt.xticks(bins,['Jan','Feb','Mar','Apr','may','Jun','July','Aug','Sep','Oct','Nov','Dec'])
        plt.xlabel("Month")
        plt.ylabel("Ratio")
        plt.show()
        

In [ ]:
data_file = ['../working//Washington-2016-Summary.csv', '../working//Chicago-2016-Summary.csv', '../working//NYC-2016-Summary.csv']
for datafile in data_file:
    total_trips_monthly(datafile)
    monthly_analysis(datafile)
    ratio(datafile)

<a id='conclusions'></a>
## Conclusions

**Question 7**: Putting the bike share data aside, think of a topic or field of interest where you would like to be able to apply the techniques of data science. What would you like to be able to learn from your chosen subject?

**Answer**: 
-  I would like to apply this technique in the field of sales. Everyday we buy some domestic products and if we already know that which product is more demandable at the current time. we can easily find good products for our use and no one will fall for the advertisement. Using this we can identify which product is more popular in a particular month.
<br>
-  I would like to learn predictions using machine learning. Because machine learning products are being used daily, perhaps without realizing it. It’s just that machine learning career is exploding now because of smart algorithms being used everywhere from email to mobile apps to marketing campaigns. I think it will give a new aim to the my career. 